# 1. Import required library

In [ ]:
import time
from datetime import datetime

import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import seaborn as sns
import numpy as np

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential, layers
from tensorflow.keras.layers import Input,SimpleRNN
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.losses import MeanSquaredError as MSELoss
from tensorflow.keras.metrics import MeanAbsolutePercentageError as MAPEMetrics
from tensorflow.keras.metrics import MeanAbsoluteError as MAEMetrics
from tensorflow.keras.metrics import MeanSquaredError as MSEMetrics
from tensorflow.keras.metrics import R2Score

# 2. Load data from csv file

In [ ]:
df = pd.read_csv('data/OPCUA_PM25_interpolated.csv', names=['Date', 'Temperature', 'Humidity', 'PM03', 'PM05', 'PM1', 'PM25', 'PM5', 'PM10'], header=0)
df_idx = df.copy(deep=True)
df_idx = df_idx.set_index(['Date'])
df_idx.index = pd.to_datetime(df_idx.index)
df_idx = df_idx.loc['2023-06-02 01:00:00':'2023-06-19 23:00:00']

## 3. Check data statistics

In [ ]:
print('Data head: ', df_idx.head(), '\nData tail: ', df_idx.tail(), '\nData shape: ', df_idx.shape, '\nData Describe: ', df_idx.describe())

## 4. Plot the data

In [ ]:
plt.figure(num=None, figsize=(24, 6), dpi=80, facecolor='w', edgecolor='k')
plt.plot(df_idx)

# 5. Inspect each data features

In [ ]:
def inspectData(data):
    checkNull = data.isnull().any()
    lengthData = len(data)
    maxVal = data.max()
    minVal = data.min()
    varVal = data.var()
    stdVal = data.std()
    meanVal = data.mean()
    medianVal = data.median()
    print('Data nullities: ', checkNull, ' | Data length: ', lengthData)
    print('Data max value: ', maxVal, ' | Data min value: ', minVal)
    print('Data variance: ', varVal, ' | Data standard deviation: ', stdVal)
    print('Data mean: ', meanVal, ' | Data median: ', medianVal)
    plt.figure(num=None, figsize=(24, 6), dpi=80, facecolor='w', edgecolor='k')
    plt.plot(data)

In [ ]:
for column in df_idx:
    print(f'[INFO] {column}')
    inspectData(df_idx[column])

## 6. Detailed statistics

In [ ]:
df_idx.info()

In [ ]:
df_idx.describe()

In [ ]:
df_idx.head()

In [ ]:
df_idx.tail()

# 7. Normalize the data

In [ ]:
zScore = StandardScaler()
np_alldata = zScore.fit_transform(df_idx)

In [ ]:
df_alldata = pd.DataFrame(np_alldata, columns = ['Temperature', 'Humidity', 'PM03', 'PM05', 'PM1', 'PM25', 'PM50', 'PM10'])

In [ ]:
df_alldata.head()

In [ ]:
df_alldata.describe()

In [ ]:
for column in df_alldata:
    print(f'[INFO] {column}')
    inspectData(df_alldata[column])

# 8. Inspect feature importance

In [ ]:
sns.set(rc={'figure.figsize':(15,9)})
sns.heatmap(df_alldata.corr(),vmin=0,vmax=1,annot=True)

# 9. Drop unimportant feature

In [ ]:
df_alldata = df_alldata.drop(['Temperature', 'Humidity', 'PM03', 'PM10'], axis=1)

In [ ]:
df_alldata.head()

In [ ]:
df_alldata.describe()

# 10. Develop RNN model

## 10.1. Specify training parameters

In [ ]:
val_length = 0.2
lr = 5e-8
labels_length = 3660
seq_length = 60
data_features = 4
batch_size = 128
epochs = 100
checkpoint_cb_monitor = 'val_loss'
earlystop_cb_monitor = 'loss'
earlystop_cb_minDelta = 0.000001
earlystop_cb_patience = 100
reduceLR_cb_monitor = 'val_loss'
reduceLR_cb_factor = 0.95
reduceLR_cb_patience = 100
reduceLR_cb_minDelta = 0.000001

## 10.2. Split into train and val data

In [ ]:
train_size = int((len(df_alldata) - labels_length) * (1 - val_length))
val_size = int((len(df_alldata) - labels_length) - train_size)
print('\nData length: ', len(df_alldata), '\nTrain data size: ', train_size, '\nVal data size: ', val_size)

df_train = pd.DataFrame(df_alldata.iloc[0:train_size, :])
df_valid = pd.DataFrame(df_alldata.iloc[train_size:train_size+val_size, :])
df_test = pd.DataFrame(df_alldata.iloc[train_size+val_size:, :])

print('\nData length: ', len(df_train), '\nData head: ', df_train.head(), '\nData tail: ', df_train.tail())
print('\nData length: ', len(df_valid), '\nData head: ', df_valid.head(), '\nData tail: ', df_valid.tail())

## 10.3. Convert to sliding window

In [ ]:
def create_dataset (X, y, look_back = 1):
    Xs, ys = [], []
 
    for i in range(0,len(X)-look_back):
        v = X[i:i+look_back]
        w = y[i+look_back]
        Xs.append(v)
        ys.append(w)
 
    return np.array(Xs), np.expand_dims(np.array(ys), axis=1)

In [ ]:
np_train = df_train.to_numpy()
np_valid = df_valid.to_numpy()
np_test = df_test.to_numpy()

X_train, y_train = create_dataset(np_train,np_train[:,2], seq_length)
X_valid, y_valid = create_dataset(np_valid,np_valid[:,2], seq_length)
X_test, y_test = create_dataset(np_test,np_test[:,2], seq_length)

In [ ]:
print(X_train.shape)
print(y_train.shape)
print(X_valid.shape)
print(y_valid.shape)

## 10.4. Create the model

In [ ]:
def create_model(units):
    model = Sequential()
    model.add(Input(shape=(seq_length, data_features)))
    model.add(SimpleRNN(units = units*2, return_sequences=True))
    model.add(SimpleRNN(units = units*5, return_sequences=True))
    model.add(SimpleRNN(units = 1))

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
                  loss='mse',
                  metrics=[MAEMetrics(), MSEMetrics(), MAPEMetrics(), tf.nn.log_poisson_loss, R2Score])
    return model

model_build = create_model(seq_length)

In [ ]:
model_build.summary()

## 10.5. Define optimization

In [ ]:
checkpoint_path = 'model_store/best_model.weights.h5'

cp_callback = ModelCheckpoint(checkpoint_path, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=True)
earlystop_callback = EarlyStopping(monitor='loss', min_delta=0.000001, patience=20)
hist_callback = tf.keras.callbacks.History()
reduceLR_callback = ReduceLROnPlateau(monitor='val_loss', factor=0.005, patience=10, verbose=1, min_delta=0.000001)

## 10.6. Train the model

In [ ]:
def fit_model(model):
    history = model.fit(X_train, y_train, epochs=epochs,  
                        validation_data=(X_valid, y_valid),
                        batch_size=batch_size, shuffle=True,
                        callbacks=[cp_callback, hist_callback, earlystop_callback, reduceLR_callback])
#                         callbacks=[cp_callback, hist_callback, reduceLR_callback])
    return history

In [ ]:
start_time = time.time()
st_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
st_time2 = datetime.now().strftime('%Y%m%d %H%M%S')
print('Start time: ', datetime.now())

train_history = fit_model(model_build)

print('Start time: ', st_time, '\nFinished time: ', datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
print('Overall training time: ', (time.time()-start_time)/3600, ' hours')

## 10.7. Plot training loss

In [ ]:
def plot_loss (history):
    fig, ax= plt.subplots(figsize = (10, 6))
    ax.set_xlim(0, len(history.history['loss'])-1)
#     ax.set_ylim(0, 1)
    ax.tick_params(axis="y",direction="in")
    ax.tick_params(axis="x",direction="in")
    ax.xaxis.set_major_locator(MultipleLocator(0.1*len(history.history['loss'])))
#     ax.yaxis.set_major_locator(MultipleLocator(.2))
    
    plt.plot(history.history['loss'],linewidth=2)
    plt.plot(history.history['val_loss'],linewidth=2)
    csfont = {'fontname':'Times New Roman'}
    plt.title('PM25 Prediction using RNN')
    plt.ylabel('Loss',fontsize=14,**csfont)
    plt.xlabel('epoch',fontsize=14,**csfont)
    plt.legend(['Train loss', 'Validation loss'], loc='upper right',fontsize=12)

plot_loss(train_history)

# 11. Test the trained model

## 11.1. Load the best model

In [ ]:
model_build.load_weights(checkpoint_path)

## 11.2. Perform the prediction using the trained model

In [ ]:
prediction_result = model_build.predict(X_test)


## 11.3. Plot the prediction results

In [ ]:
def plot_future(prediction, y_test):
    fig, ax= plt.subplots(figsize = (12, 8))
    ax.set_xlim(0, y_test.shape[0])
    ax.tick_params(axis="y",direction="in")
    ax.tick_params(axis="x",direction="in")
    ax.xaxis.set_major_locator(MultipleLocator(0.1*y_test.shape[0]))
    csfont = {'fontname':'Times New Roman'}
    range_future = len(prediction)
    plt.plot(np.arange(range_future), np.array(y_test),  label='Test data',linewidth=2)
    plt.plot(np.arange(range_future),np.array(prediction),label='Prediction',linewidth=2)
    plt.legend(loc='upper left',fontsize=14)
    plt.xlabel('Time (second)',fontsize=16,**csfont)
    plt.ylabel('PM2.5 (µg/m3)',fontsize=16,**csfont)
    
plot_future(prediction_result, y_test)